In [3]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.momentum import RSIIndicator,StochasticOscillator
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange
from ta.volume import OnBalanceVolumeIndicator
import warnings
warnings.filterwarnings('ignore')

In [4]:
def fetch_nifty_data(ticker="^NSEI", start_date="2000-01-01", end_date="2025-08-19"):
    df = yf.download(ticker, start=start_date, end=end_date)
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

# Step 2: Calculate technical indicators
def calculate_technical_indicators(df):
    # Simple Moving Average (SMA)
    li = df['Close'].to_numpy().flatten()
#    df['SMA_20'] = SMAIndicator(li, window=20).sma_indicator()
    df['SMA_20'] = SMAIndicator(df['Close']['^NSEI'], window=20).sma_indicator()
    df['SMA_50'] = SMAIndicator(df['Close']['^NSEI'], window=50).sma_indicator()
    
    # Exponential Moving Average (EMA)
    df['EMA_12'] = EMAIndicator(df['Close']['^NSEI'], window=12).ema_indicator()
    df['EMA_26'] = EMAIndicator(df['Close']['^NSEI'], window=26).ema_indicator()
    
    # Relative Strength Index (RSI)
    df['RSI'] = RSIIndicator(df['Close']['^NSEI'], window=14).rsi()
    
    # MACD
    macd = MACD(df['Close']['^NSEI'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    
    # Bollinger Bands
    bb = BollingerBands(df['Close']['^NSEI'], window=20)
    df['BB_High'] = bb.bollinger_hband()
    df['BB_Low'] = bb.bollinger_lband()
    
    df['ATR'] = AverageTrueRange(df['High']['^NSEI'], df['Low']['^NSEI'], df['Close']['^NSEI'], window=14).average_true_range()
    
    stochastic = StochasticOscillator(df['High']['^NSEI'], df['Low']['^NSEI'], df['Close']['^NSEI'], window=14, smooth_window=3)
    df['Stoch_K'] = stochastic.stoch()
    df['Stoch_D'] = stochastic.stoch_signal()
    df['OBV'] = OnBalanceVolumeIndicator(df['Close']['^NSEI'], df['Volume']['^NSEI']).on_balance_volume()
    
    return df

def fetch_USDINR_data(ticker="INR=X", start_date="2000-01-01", end_date="2025-08-19"):
    df = yf.download(ticker, start=start_date, end=end_date)
    return df[['Close']]

def fetch_IndiaVIX_data(ticker="^INDIAVIX", start_date="2000-01-01", end_date="2025-08-19"):
    df = yf.download(ticker, start=start_date, end=end_date)
    return df[['Close']]

def fetch_crudeoil_data(ticker="CL=F", start_date="2000-01-01", end_date="2025-08-19"):
    df = yf.download(ticker, start=start_date, end=end_date)
    return df[['Close']]

def fetch_SP500_data(ticker="^GSPC", start_date="2000-01-01", end_date="2025-08-19"):
    df = yf.download(ticker, start=start_date, end=end_date)
    return df[['Close']]


def add_macro_indicators():

    india_vix = fetch_IndiaVIX_data()['Close']
    fx_rate = fetch_USDINR_data()['Close']
    oil = fetch_crudeoil_data()['Close']
    spx = fetch_SP500_data()['Close']
    return india_vix,fx_rate,oil,spx

In [5]:
india_vix,fx_rate,oil,spx = add_macro_indicators()
macro_data = pd.concat([india_vix,fx_rate,oil,spx], axis=1)
nifty_data = fetch_nifty_data()
technical_data = calculate_technical_indicators(nifty_data)
input_data = pd.concat([technical_data,macro_data],axis = 1)
input_data = input_data.dropna()
input_data.columns = ['NSEI_Open','NSEI_High','NSEI_Low','NSEI_Close','NSEI_Volume','SMA_20','SMA_50','EMA_12','EMA_26','RSI','MACD','MACD_Signal','BB_High','BB_Low','ATR','Stoch_K','Stoch_D','OBV','INDIAVIX','FX_Rate','Oil','SP500']
input_data = input_data[['SMA_20','SMA_50','EMA_12','EMA_26','RSI','MACD','MACD_Signal','BB_High','BB_Low','ATR','Stoch_K','Stoch_D','OBV','INDIAVIX','FX_Rate','NSEI_Close','Oil','SP500']]
input_data['NIFTY_Next'] = input_data['NSEI_Close'].shift(-1)
input_data['Target'] = (input_data['NIFTY_Next'] > input_data['NSEI_Close']).astype(int)
del input_data['NSEI_Close']
del input_data['NIFTY_Next']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [71]:
features = ['SMA_20',
         #   'SMA_50',
            'EMA_12',
         #   'EMA_26',
            'RSI',
            'MACD',
   #         'MACD_Signal',
             'ATR',
            'INDIAVIX',
            'FX_Rate']
   #         'Oil']
X = input_data[features]
y = input_data['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Initialize and train the model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)
df_test = y_test.to_frame()
df_test['Predicted'] = y_pred
df_test= df_test.reset_index()
for i in range(0,len(df_test)):
    if df_test.loc[i,'Target']==df_test.loc[i,'Predicted']:
        df_test.loc[i,'Accurate'] = 1
    else:
        df_test.loc[i,'Accurate'] = 0

In [72]:
accuracy = 100*(df_test['Accurate'].sum()/len(df_test))
accuracy

np.float64(50.96852300242131)

In [73]:
df_test['Actual_Date'] = df_test['Date'].shift(-1)
df_test = df_test[['Actual_Date','Target','Predicted','Accurate']]
#df_test
nifty_test = yf.download('^NSEI',start = df_test.loc[0,'Actual_Date'], end = df_test.loc[len(df_test)-2,'Actual_Date'])
nifty_test = nifty_test['Close']['^NSEI']
nifty_test = nifty_test.to_frame().reset_index()
nifty_test.columns = ['Actual_Date','NIFTYClosing']
df_test = df_test.merge(nifty_test,on=['Actual_Date'],how='left')
#df_test
portfolio = 100000
for i in range(1,len(df_test)-2):
    if df_test.loc[i-1,'Predicted']==0:
        sell = df_test.loc[i-1,'NIFTYClosing']
        buy = df_test.loc[i,'NIFTYClosing'] 
    else:
        sell = df_test.loc[i,'NIFTYClosing']
        buy = df_test.loc[i-1,'NIFTYClosing']
    df_test.loc[i,'Returns'] = (sell/buy)-1
    portfolio = portfolio*(sell/buy)

[*********************100%***********************]  1 of 1 completed


In [75]:
df_test['Returns'].describe()

count    823.000000
mean       0.000807
std        0.008618
min       -0.036779
25%       -0.004431
50%        0.000335
75%        0.005619
max        0.063031
Name: Returns, dtype: float64

In [74]:
portfolio

np.float64(188421.52906728315)

In [77]:
sharpe = (252*0.000807-0.06)/(np.sqrt(252)*0.008618)
sharpe

np.float64(1.0479325966677273)